In [5]:
import requests
from bs4 import BeautifulSoup
from urllib.error import HTTPError
from urllib import request
import pandas as pd
import os
import psycopg2
from PIL import Image

#파일다운로드 함수
def get_download(url, fname, directory):
   try:
       os.chdir(directory)
       request.urlretrieve(url, fname)
       print('다운로드 완료\n')
   except HTTPError as e:
       print('error')
       return 


#목록 조회 html 크롤링#
totalPage = 2392
basicUrl = "https://www.csi.go.kr"
listUrl = f"{basicUrl}/acd/acdCaseList.do"    #목록조회url
detailUrl = f"{basicUrl}/acd/acdCaseView.do"  #상세조회url
picUrl = f"{basicUrl}/acd/photoInc.do"        #상세-사진조회url

#페이징 별로 목록 조회#
for i in range(1,totalPage+1):
     data = []  # 상세조회 데이터
     listPostResponse =requests.post(f"{listUrl}?page_count={i}&biz_id=acdDB&search_key=1")
     listHtml = listPostResponse.text
     listSoup = BeautifulSoup(listHtml,'html.parser')
     items = listSoup.select('tbody>.odd.gradeX>.t-left>a')
     for item in items :
        #case_no = 26275   # 특정 사건번호 테스트 시 사용
        #case_no = item.attrs['href'][21:26]
        case_no = item.attrs['href'].split("'")[1]
        print(case_no)

        #각 사건 번호로 상세 조회 크롤링#
        #상세 조회 html 크롤링#
        detailResponse = requests.get(f"{detailUrl}?case_no={case_no}")
        detailHtml = detailResponse.text
        datailSoup = BeautifulSoup(detailHtml,'html.parser')

        #hidden value#
        biz_id = datailSoup.find("input", {"id": "biz_id"}).get('value')
        #accd_no = '202300007548'
        accd_no = datailSoup.find("input", {"id": "accd_no"}).get('value')
        menu_id = datailSoup.find("input", {"id": "menu_id"}).get('value')
        start_ymd = datailSoup.find("input", {"id": "start_ymd"}).get('value')
        end_ymd = datailSoup.find("input", {"id": "end_ymd"}).get('value')
        search_key = datailSoup.find("input", {"id": "search_key"}).get('value')
        search_val = datailSoup.find("input", {"id": "search_val"}).get('value')
        page_count = datailSoup.find("input", {"id": "page_count"}).get('value')
        
        #hidden value로 사진 조회 작업#
        picPayLoad = {
            "biz_id": biz_id,
            "accd_no": accd_no,
            "menu_id": menu_id,
            "start_ymd": start_ymd,
            "end_ymd": end_ymd,
            "search_key": search_key,
            "search_val": search_val,
            "page_count": page_count
        }
        picResponse = requests.post(picUrl,data=picPayLoad)
        picHtml = picResponse.text
        picSoup = BeautifulSoup(picHtml,'html.parser')
        
        
        # 필요한 데이터 추출 및 값 저장
        # 상세 데이터 추출용 tr태그들만 select
        detailData = datailSoup.select('.contentsRow>.table.table-bordered>tbody>tr>.t-left')
        
        #DB 저장할데이터 추출#
        # 사고명,발생일시,공공/민간구분,기상상태,시설물 종류,사고유형 - 인적사고, 사고유형 - 물적사고,사고분류-공종,사고분류-사고객체,사고분류-작업프로세스,사고경위,사고원인,구체적 사고원인,피해상황-사망자수,피해상황-부상자수,피해상황-피해금액,피해상황-피해내용,재발방지대책,
        
        #공공민간 구분 = column1
        column1 = detailData[3].text.strip()
        #발생일시 - column2
        column2 = detailData[1].text.strip()
        # 사고명 - column3
        column3 = detailData[0].text.strip()
        #시설물종류_구분1 - column4
        sago_ty= detailData[5].text.split()
        column4 =sago_ty[0]
        #시설물종류_구분2 - column5
        column5 = sago_ty[2]
        #시설물종류_구분3 - column6
        if len(sago_ty) > 5 :
            column6 = ''.join(sago_ty[4:])
        else : column6=' '
        
        #날씨 - column7
        weather = detailData[4].select('label')
        column7 = weather[0].text.split()[2]
        #기온 - column8
        column8 = weather[1].text.split()[2][:2]
        #습도 - column9
        column9 = weather[2].text.split()[2][:2]
        #사고유형_인적사고 - column10
        column10= detailData[6].text.strip()
        #사고유형_물적사고 - column11
        column11= detailData[8].text.strip()
        #공종_구분1 - column12, 공종_구분2 - column13
        intermission_ty= detailData[9].text
        if '>' in intermission_ty : 
            spIntermission_ty = intermission_ty.split('>')
            column12 = spIntermission_ty[0].strip()
            column13 = spIntermission_ty[1].strip()
        else  : 
            column12 = " "
            column13 = " "
        print('intermission_ty : ', intermission_ty)
        
        #사고객체_구분1 - column14, 사고객체_구분2 - column15
        accdTarget_ty = detailData[10].text
        if '>' in accdTarget_ty : 
            spAccdTarget_ty = accdTarget_ty.split('>')
            column14 = spAccdTarget_ty[0].strip()
            column15 = spAccdTarget_ty[1].strip()
        else : 
            column14 = " "
            column15 = " "

        #작업프로세스 - column16
        column16 = detailData[11].text.strip()
        #사고경위 - column17
        column17 =  detailData[14].text.strip()
        #사고원인 - column18
        accd_cause = detailData[15].text.split()
        column18 = ''.join(accd_cause[3:])
        #구체적사고원인 - column19
        column19 = detailData[16].text.strip()
        #사망자수 - column20
        deadCnt = detailData[17].select('.form-inline > div')
        deadCnt_1 = int(deadCnt[0].text.split()[2][:1])
        deadCnt_2 = int(deadCnt[1].text.split()[2][:1])
        column20 = deadCnt_1+deadCnt_2
        #부상자수 - column21
        injuredCnt = detailData[18].select('.form-inline>div')
        injuredCnt_1 = int(injuredCnt[0].text.split()[2][:1])
        injuredCnt_2 = int(injuredCnt[1].text.split()[2][:1])
        column21 = injuredCnt_1+injuredCnt_2

        #피해금액 - column22
        column22 = detailData[19].text.strip()
        #피해내용 - column23
        column23 = detailData[20].text.strip()
        #재발방지대책 - column24
        column24 = detailData[23].text.strip()
        #사진ID - column25

        # 사진 데이터 추출
        picInfo = picSoup.select_one('.contentsRow')
        if picInfo!= None : 
            pictures = picSoup.select('tbody>tr>td>img')
            column25 = accd_no #사진 조회 번호#
            if len(pictures)>=2 :
                #print(f'사진 {len(pictures)}개 있다! ')
                for picture in pictures :
                    print(picture)
                    pic_src = picture.attrs['src']
                    pic_name= picture.attrs['data-filename']
                    #print(f'{basicUrl}{pic_src}')
                    fileUrl = f'{basicUrl}{pic_src}'
                    filePath = "D:\\nes\\safetytalk-web\\home\\awa-safety\\opt\\safetytalk\\safetytalk-upload\\sago"
                    get_download(fileUrl,pic_name,filePath)
                    img = Image.open(pic_name)
                    print(img.format)
                
                    #print(pic_name,pic_src)
        else :
            column25 = ''
        
        print('페이지번호 : ', i ,'사건번호 : ',case_no, column1,column2,column3,column4,column5,column6,column7,column8,column9,column10,column11,column12,column13,column14,column15,column16,column17,column18,column19,column20,column21,column22,column23,column24,column25)
        data.append([column1,column2,column3,column4,column5,column6,column7,column8,column9,column10,column11,column12,column13,column14,column15,column16,column17,column18,column19,column20,column21,column22,column23,column24,column25])
    # print(data,'\n',len(data))
    # columnName = ["column1","column2","column3","column4","column5","column6","column7","column8","column9","column10","column11","column12","column13","column14","column15","column16","column17","column18","column19","column20","column21","column22","column23","column24","column25"]
    # insertDB(self=db , schema="fsms",table="com_sago_temp",
    #          colum=["column1","column2","column3","column4","column5","column6","column7","column8","column9","column10","column11","column12","column13","column14","column15","column16","column17","column18","column19","column20","column21","column22","column23","column24","column25"],
    #          data=data)

#엑셀 작업#
#데이터 프레임 만들기
#df = pd.DataFrame(data,columns=['공공민간구분','발생일시','사고명','시설물종류_구분1','시설물종류_구분2','시설물종류_구분3','날씨','기온','습도','사고유형_인적사고','사고유형_물적사고','공종_구분1','공종_구분2','사고객체_구분1','사고객체_구분2','작업프로세스','사고경위','사고원인','구체적사고원인','사망자수','부상자수','피해금액','피해내용','재발방지대책','사진ID'])

#엑셀저장
#df.to_excel('CSI 크롤링 엑셀 데이터.xlsx')

27514
intermission_ty :  건축 > 도장공사
페이지번호 :  1 사건번호 :  27514 공공 2024-05-20 오전 11:00 대구 친환경발전소 건설공사관련 도장작업중 말비계하부 떨어짐 재해 산업환경설비 발전시설   맑음 29 50 떨어짐(2미터 미만) 없음 건축 도장공사 가시설 기타 가시설 도장작업 작업 마무리를 하고 말비계에서 내려오던 중 발을 미끌리며 왼쪽발을 접질려 착지 중 사고 발생(왼쪽 발 뒤꿈치 실금) 작업자부주의 작업 마무리를 하고 말비계에서 내려오던 중 발을 미끌리며 왼쪽발을 접질려 착지 중 사고 발생(작업자 부주의) 0 1 피해없음 없음 작업자 안전교육 철저, 위험성평가 반영 등 202400002384
27500
intermission_ty :  건축 > 목공사
<img align="absmiddle" alt="" border="0" data-filename="KakaoTalk_20240517_135635714_01.jpg" onerror="this.src='/com/img/noimg.jpg'" src="/com/imageViewProc.do?file_no=5LwwQRAtD0ZWSpi83PRQ+w==" style="width:100%;"/>
다운로드 완료

JPEG
<img align="absmiddle" alt="" border="0" data-filename="KakaoTalk_20240517_145914438.jpg" onerror="this.src='/com/img/noimg.jpg'" src="/com/imageViewProc.do?file_no=PyictN06mseuI4VF+OAWkQ==" style="width:100%;"/>
다운로드 완료

JPEG
<img align="absmiddle" alt="" border="0" data-filename="KakaoTalk_20240517_145914438_01.jpg" onerror="this.src='/com/img/noimg.jpg'" src="/com/ima

KeyboardInterrupt: 